In [19]:
# ============================================
# STEP 1 — Install dependencies
# ============================================
!pip install pillow pandas openpyxl requests

import requests

font_url = "https://github.com/google/fonts/raw/main/ofl/audiowide/Audiowide-Regular.ttf"
font_path = "Audiowide-Regular.ttf"

response = requests.get(font_url)
with open(font_path, "wb") as f:
    f.write(response.content)

print("✅ Audiowide TTF downloaded")




[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


✅ Audiowide TTF downloaded


In [ ]:
# ============================================
# STEP 2 — Upload required files (Colab only)
# ============================================

def running_in_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False


if running_in_colab():
    from google.colab import files
    
    print("📂 Please upload the following files:")
    print(" - lists.xlsx")
    print(" - forbidden.png")
    print(" - mimic.png")
    print(" - drawit.png")

    uploaded = files.upload()
    print("\nUploaded files:", list(uploaded.keys()))
else:
    print("💻 Running locally — make sure required files are in the project folder.")

💻 Running locally — make sure required files are in the project folder.


In [25]:
# ============================================
# IMPORT REQUIRED LIBRARIES
# ============================================

import pandas as pd                            # For reading Excel files
from PIL import Image, ImageDraw, ImageFont    # For image editing
from IPython.display import display            # For displaying images in Colab
import textwrap                                # For wrapping long text
import os                                      # For folder creation
import shutil                                  # For zipping folders


# ============================================
# CONFIGURATION
# ============================================

LIST_NAME = "list1"                            # Excel sheet name
TEMPLATE_F = "forbidden.png"                   # Background template image
TEMPLATE_M = "mimic.png"                   # Background template image
TEMPLATE_D = "drawit.png"                   # Background template image
EXCEL_FILE = "lists.xlsx"                      # Excel file with words
FONT_PATH = "/content/Audiowide-Regular.ttf"   # Font file path
OUTPUT_FOLDER_F = "Tokacards/forbidden"          # Output folder name
OUTPUT_FOLDER_M = "Tokacards/mimeit"          # Output folder name
OUTPUT_FOLDER_D = "Tokacards/drawit"          # Output folder name


# ============================================
# CREATE OUTPUT FOLDER
# ============================================

# Create folder if it does not already exist
os.makedirs(OUTPUT_FOLDER_F, exist_ok=True)
os.makedirs(OUTPUT_FOLDER_M, exist_ok=True)
os.makedirs(OUTPUT_FOLDER_D, exist_ok=True)

# ============================================
# LOAD EXCEL DATA
# ============================================

# Read the specified sheet from the Excel file
df = pd.read_excel(EXCEL_FILE, sheet_name=LIST_NAME)


# ============================================
# LOOP THROUGH EACH ROW (ONE CARD PER ROW) - FORBIDDEN
# ============================================

for i in range(len(df)):

    # Extract data from Excel row
    word = str(df.loc[i, "word"])

    # HANDLE HYPHENS IN MAIN WORD WITH SPECIFIC RULES:
    # If "-" has space before and after, split into new line
    # If no spaces around "-", keep it as is
    if " - " in word:  # Check for hyphen with spaces on both sides
        # Split at " - " (hyphen with spaces)
        word_parts = word.split(" - ")
        # Remove empty strings and strip whitespace
        word_parts = [part.strip() for part in word_parts if part.strip()]
        # Join with newline (removing the hyphen and spaces)
        word = "\n".join(word_parts)
    # If "-" without spaces (like in compound words), leave it as is

    desc = str(df.loc[i, "description"])
    forbidden = [str(df.loc[i, f"forbidden{n}"]) for n in range(1, 6)]

    # Load template image
    img = Image.open(TEMPLATE_F).convert("RGBA")
    W, H = img.size                             # Image width & height
    draw = ImageDraw.Draw(img)

    # ----------------------------------------
    # Helper function: normalized coordinates
    # ----------------------------------------
    def norm(x, y):
        """Convert normalized (0–1) coords to pixel coords"""
        return (x * W, y * H)

    # ----------------------------------------
    # Helper function: draw wrapped text
    # ----------------------------------------
    def draw_wrapped(text, xy, font_size, width=25):
        """
        Draw text with automatic wrapping
        """
        # MODIFIED: Preserve existing newlines, only wrap lines that are too long
        lines = text.split('\n')
        wrapped_lines = []
        for line in lines:
            # Only wrap if line is longer than width
            if len(line) > width:
                wrapped_lines.extend(textwrap.wrap(line, width=width))
            else:
                wrapped_lines.append(line)

        wrapped = "\n".join(wrapped_lines)
        font = ImageFont.truetype(font_path, font_size)
        draw.text(xy, wrapped, fill="white", font=font, anchor="mm", align="center")

    # ----------------------------------------
    # DRAW TEXT ON CARD
    # ----------------------------------------

    # Main word (top)
    draw_wrapped(word, norm(0.5, 0.1), font_size=40, width=25)

    # Description
    draw_wrapped(desc, norm(0.5, 0.25), font_size=25, width=40)

    # Forbidden words list
    ys = [0.49, 0.56, 0.63, 0.70, 0.77]
    for y, text in zip(ys, forbidden):
        draw_wrapped(text, norm(0.5, y), font_size=40, width=25)

    # ----------------------------------------
    # SAVE OUTPUT IMAGE
    # ----------------------------------------

    output_name = f"forbidden_{i+1}.png"
    output_path = os.path.join(OUTPUT_FOLDER_F, output_name)
    img.save(output_path)

    # Display image in notebook
    #display(img)

    #print(f"✅ Card {i+1} generated successfully")


# ============================================
# LOOP THROUGH EACH ROW (ONE CARD PER ROW) - MIME IT
# ============================================

for i in range(len(df)):

    # Extract data from Excel row
    word = str(df.loc[i, "word"])

    # HANDLE HYPHENS IN MAIN WORD WITH SPECIFIC RULES:
    # If "-" has space before and after, split into new line
    # If no spaces around "-", keep it as is
    if " - " in word:  # Check for hyphen with spaces on both sides
        # Split at " - " (hyphen with spaces)
        word_parts = word.split(" - ")
        # Remove empty strings and strip whitespace
        word_parts = [part.strip() for part in word_parts if part.strip()]
        # Join with newline (removing the hyphen and spaces)
        word = "\n".join(word_parts)
    # If "-" without spaces (like in compound words), leave it as is

    desc = str(df.loc[i, "description"])

    # Load template image
    img = Image.open(TEMPLATE_M).convert("RGBA")
    W, H = img.size                             # Image width & height
    draw = ImageDraw.Draw(img)

    # ----------------------------------------
    # Helper function: normalized coordinates
    # ----------------------------------------
    def norm(x, y):
        """Convert normalized (0–1) coords to pixel coords"""
        return (x * W, y * H)

    # ----------------------------------------
    # Helper function: draw wrapped text
    # ----------------------------------------
    def draw_wrapped(text, xy, font_size, width=25):
        """
        Draw text with automatic wrapping
        """
        # MODIFIED: Preserve existing newlines, only wrap lines that are too long
        lines = text.split('\n')
        wrapped_lines = []
        for line in lines:
            # Only wrap if line is longer than width
            if len(line) > width:
                wrapped_lines.extend(textwrap.wrap(line, width=width))
            else:
                wrapped_lines.append(line)

        wrapped = "\n".join(wrapped_lines)
        font = ImageFont.truetype(font_path, font_size)
        draw.text(xy, wrapped, fill="white", font=font, anchor="mm", align="center")

    # ----------------------------------------
    # DRAW TEXT ON CARD
    # ----------------------------------------

    # Main word (top)
    draw_wrapped(word, norm(0.5, 0.1), font_size=40, width=25)

    # Description
    draw_wrapped(desc, norm(0.5, 0.25), font_size=25, width=40)

    # ----------------------------------------
    # SAVE OUTPUT IMAGE
    # ----------------------------------------

    output_name = f"mimeit_{i+1}.png"
    output_path = os.path.join(OUTPUT_FOLDER_M, output_name)
    img.save(output_path)

    # Display image in notebook
    #display(img)

    #print(f"✅ Card {i+1} generated successfully")

# ============================================
# LOOP THROUGH EACH ROW (ONE CARD PER ROW) - MIME IT
# ============================================

for i in range(len(df)):

    # Extract data from Excel row
    word = str(df.loc[i, "word"])

    # HANDLE HYPHENS IN MAIN WORD WITH SPECIFIC RULES:
    # If "-" has space before and after, split into new line
    # If no spaces around "-", keep it as is
    if " - " in word:  # Check for hyphen with spaces on both sides
        # Split at " - " (hyphen with spaces)
        word_parts = word.split(" - ")
        # Remove empty strings and strip whitespace
        word_parts = [part.strip() for part in word_parts if part.strip()]
        # Join with newline (removing the hyphen and spaces)
        word = "\n".join(word_parts)
    # If "-" without spaces (like in compound words), leave it as is

    desc = str(df.loc[i, "description"])

    # Load template image
    img = Image.open(TEMPLATE_D).convert("RGBA")
    W, H = img.size                             # Image width & height
    draw = ImageDraw.Draw(img)

    # ----------------------------------------
    # Helper function: normalized coordinates
    # ----------------------------------------
    def norm(x, y):
        """Convert normalized (0–1) coords to pixel coords"""
        return (x * W, y * H)

    # ----------------------------------------
    # Helper function: draw wrapped text
    # ----------------------------------------
    def draw_wrapped(text, xy, font_size, width=25):
        """
        Draw text with automatic wrapping
        """
        # MODIFIED: Preserve existing newlines, only wrap lines that are too long
        lines = text.split('\n')
        wrapped_lines = []
        for line in lines:
            # Only wrap if line is longer than width
            if len(line) > width:
                wrapped_lines.extend(textwrap.wrap(line, width=width))
            else:
                wrapped_lines.append(line)

        wrapped = "\n".join(wrapped_lines)
        font = ImageFont.truetype(font_path, font_size)
        draw.text(xy, wrapped, fill="white", font=font, anchor="mm", align="center")

    # ----------------------------------------
    # DRAW TEXT ON CARD
    # ----------------------------------------

    # Main word (top)
    draw_wrapped(word, norm(0.5, 0.1), font_size=40, width=25)

    # Description
    draw_wrapped(desc, norm(0.5, 0.25), font_size=25, width=40)

    # ----------------------------------------
    # SAVE OUTPUT IMAGE
    # ----------------------------------------

    output_name = f"drawit_{i+1}.png"
    output_path = os.path.join(OUTPUT_FOLDER_D, output_name)
    img.save(output_path)

    # Display image in notebook
    #display(img)

    #print(f"✅ Card {i+1} generated successfully")



# ============================================
# ZIP THE OUTPUT FOLDER
# ============================================

zip_name = "Tokacards"
shutil.make_archive(zip_name, 'zip', OUTPUT_FOLDER)

print("📦 Tokacards folder zipped successfully")


📦 Tokacards folder zipped successfully
